In [ ]:
! wget http://dlib.net/files/mmod_human_face_detector.dat.bz2
!bzip2 -dk mmod_human_face_detector.dat.bz2
%rm mmod_human_face_detector.dat.bz2

--2021-09-05 09:03:07--  http://dlib.net/files/mmod_human_face_detector.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694709 (678K)
Saving to: ‘mmod_human_face_detector.dat.bz2’

mmod_human_face_det 100%[===================>] 678.43K  --.-KB/s    in 0.07s   

2021-09-05 09:03:07 (10.1 MB/s) - ‘mmod_human_face_detector.dat.bz2’ saved [694709/694709]



In [ ]:
import sys, os
import numpy as np
from math import *
import math
from PIL import Image
from matplotlib import pyplot
from matplotlib.patches import Rectangle, Circle
import matplotlib.pyplot as plt
import cv2
import random
import dlib
import numpy as np

In [ ]:
def bb_intersection_over_union(box1, box2):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(box1[0], box2[0])
	yA = max(box1[1], box2[1])
	xB = min(box1[2], box2[2])
	yB = min(box1[3], box2[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
	box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(box1Area + box2Area - interArea)
	# return the intersection over union value
	return iou

In [ ]:
def calculate_iou(gtbox,MMODbox):
  no_counter = 0
  text = img_file + "\n" + str(num_faces) + "," + str(len(MMODbox)) + "\n"
  for boxA in gtbox:
    if len(MMODbox)==0:
      text += ','.join([str(elem) for elem in boxA]) + '/' + "None" + '/' + "0.0" + '/' + "0.0" + '\n'
      no_counter += 1
    else:
      test_iou = []
      for boxB in MMODbox:
        #print(boxB[0])
        test_iou.append(bb_intersection_over_union(boxA, boxB[0]))
      #FN
      if max(test_iou) == 0.0:
        text += ','.join([str(elem) for elem in boxA]) + "/" + "None" + '/' + "0.0" + "/" + str(max(test_iou)) + "\n"
        no_counter += 1
      #TP
      else:
        for i,u in enumerate(test_iou):
          if u==max(test_iou):
            index=i
            text += ','.join([str(elem) for elem in boxA]) + "/" + ','.join([str(elem) for elem in MMODbox[i][0]]) + "/" + str(MMODbox[i][1]) + "/" + str(max(test_iou)) + "\n"
            break
    """#FP
    for boxC in MMODbox:
      test_iou=[]
      for boxD in gtbox:
        test_iou.append(bb_intersection_over_union(boxC, boxD))
        if max(test_iou)==0.0:
          text += 'None' + '/' + ','.join([str(elem) for elem in boxC])' + '/' + str(correct_iou) + '\n' """
  
  #print(text)
  return text

In [ ]:
folds = ["01","02","03","04","05","06","07","08","09","10"]
for fold_num in folds:

  rect_filename = '/content/drive/MyDrive/rect/FDDB-fold-{f_name}-rectList.txt'.format(f_name = fold_num)
  iou_filename = '/content/drive/MyDrive/iou/MMOD/FDDB-fold-{f_name}-MMOD-IOU.txt'.format(f_name = fold_num)

  with open(rect_filename) as f:
    lines = [line.rstrip('\n') for line in f]

  counter = 0
  f = open(iou_filename,'w')
  while counter < len(lines):
    img_file = lines[counter] 
    image = cv2.imread(img_file)
    num_faces = int(lines[counter+1])
    gtbox = []
    for j in range(num_faces):
      gt = [int(s) for s in lines[counter+j+2].split(',')]
      gtbox.append(gt)

    dnnFaceDetector=dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = dnnFaceDetector(image, 1)
    MMODrect = []
    Confidence = []
    for (i, rect) in enumerate(rects):
      x1 = rect.rect.left()
      y1 = rect.rect.top()
      x2 = rect.rect.right()
      y2 = rect.rect.bottom()
      MMOD = [x1, y1, x2, y2]
      MMODrect.append(MMOD)
      c = rect.confidence
      #n = (c- min) / (max-min)
      Confidence.append(c)
    MMODbox = list(zip(MMODrect,Confidence))

    text = calculate_iou(gtbox,MMODbox)
    
    f.write(text)
    counter = counter + num_faces + 2

  try:
    print(no_counter)
  except: 
    pass
  print(fold_num,"is done")
  f.close()

RuntimeError: ignored